In [1]:
from redisvl.extensions.router.routes import Route, RoutingConfig

In [2]:
# Define individual routes manually with metadata
politics = Route(
    name="politics",
    references=[
        "isn't politics the best thing ever",
        "why don't you tell me about your political opinions"
    ],
    metadata={"priority": 1}
)

chitchat = Route(
    name="chitchat",
    references=[
        "hello",
        "how's the weather today?",
        "how are things going?"
    ],
    metadata={"priority": 2}
)

In [3]:
routes = [politics, chitchat]

In [4]:
config = RoutingConfig(top_k=5, distance_threshold=0.5)

In [5]:
from redisvl.extensions.router.semantic import SemanticRouter
import redis

# Create SemanticRouter named "topic-router"
redis_client = redis.Redis()
routes = [politics, chitchat]
topic_router = SemanticRouter(
    name="topic-router",
    routes=routes,
    routing_config=config,
    redis_client=redis_client
)

22:39:55 redisvl.index.index INFO   Index already exists, not overwriting.


In [6]:
# Query topic-router with default behavior based on the config
result = topic_router("don't you love politics?")

In [7]:
assert topic_router.routes == routes
assert topic_router.name == "topic-router"
assert topic_router.name == topic_router._index.name == topic_router._index.prefix
assert topic_router.routing_config == config

In [8]:
config.top_k

5

In [10]:
topic_router("how are you")

[{'id': 'topic-router:chitchat:186465374219e387c415753e03e45813d5d6d1291d7d4cd78107b2d8528817b5',
  'vector_distance': '0.45913541317',
  'route_name': 'chitchat',
  'reference': 'how are things going?'}]